In [1]:
import pandas as pd
import numpy as np
from os import walk
from sklearn.preprocessing import MinMaxScaler

In [2]:
path_eco = './analyse_data/Indicators_eco/'
path_work = './analyse_data/Indicators_work/'
path_edu = './analyse_data/Indicators_education/'
file_world = './analyse_data/details_countries.csv'
path = [path_eco, path_edu, path_work]

In [21]:
df_world = pd.read_csv('./analyse_data/details_countries.csv')

In [14]:
df_world[df_world["country-code"].isnull()]

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,region-code,sub-region-code


In [4]:
file_merged = pd.DataFrame()
for _path in path:
    for filename in next(walk(_path))[2]:
        file_tmp = pd.read_excel(_path + filename, sheetname='Data')
        file_tmp['indicator'] = filename.replace('.xlsx', '')
        file_tmp['indicator2'] = file_tmp.columns.tolist()[0]
        file_tmp['name'] = file_tmp[file_tmp.columns.tolist()[0]]
        del file_tmp[file_tmp.columns.tolist()[0]]

        file_tmp = pd.melt(file_tmp, id_vars=['name', 'indicator', 'indicator2'],
                                    value_vars=list(file_tmp.columns[:-3]),
                                    var_name='year', value_name='value')
        file_merged = pd.concat([file_merged, file_tmp])

In [5]:
file_merged.head()

,name,indicator,indicator2,year,value
0,Abkhazia,Aid received,"ODA received, total (constant 2010 US$)",1960,NaN
1,Afghanistan,Aid received,"ODA received, total (constant 2010 US$)",1960,1.088000e+08
2,Akrotiri and Dhekelia,Aid received,"ODA received, total (constant 2010 US$)",1960,NaN
3,Albania,Aid received,"ODA received, total (constant 2010 US$)",1960,NaN
4,Algeria,Aid received,"ODA received, total (constant 2010 US$)",1960,3.405040e+09


In [197]:
df_world = pd.read_csv('./analyse_data/details_countries.csv')

In [198]:
file_merged_new = pd.merge(file_merged, df_world[['name', 'region', 'country-code']], on='name', how='inner')

In [199]:
len(file_merged_new)

224550

In [167]:
file_merged_new.head()

,name,indicator,indicator2,year,value,region,country-code
0,Abkhazia,Aid received,"ODA received, total (constant 2010 US$)",1960,NaN,NaN,NaN
1,Afghanistan,Aid received,"ODA received, total (constant 2010 US$)",1960,1.088000e+08,Asia,4.0
2,Akrotiri and Dhekelia,Aid received,"ODA received, total (constant 2010 US$)",1960,NaN,NaN,NaN
3,Albania,Aid received,"ODA received, total (constant 2010 US$)",1960,NaN,Europe,8.0
4,Algeria,Aid received,"ODA received, total (constant 2010 US$)",1960,3.405040e+09,Africa,12.0


In [205]:
country_to_delete = ["Abkhazia", "Akrotiri and Dhekelia", 'Channel Islands', "Faeroe Islands", 
                     "Netherlands Antilles", "Ngorno-Karabakh", "Northern Cyprus", "St. Helena", "St. Kitts and Nevis",
                    "South Ossetia", "Transnistria", "West Bank and Gaza", "Yugoslavia", "Åland", "Christian", 
                     "Coastline", "Hawaiian Trade Zone", "U.S. Pacific Islands", "Wake Island", "Sark", "Chinese Taipei",
                    "Saint Eustatius", "Saba", "Nagorno-Karabakh", "Saint Helena",
                    "Serbia and Montenegro", "West Germany", "East Germany",
                    "North Yemen (former)", "South Yemen (former)", "Eritrea and Ethiopia",
                    "Serbia excluding Kosovo"]

In [206]:
list_countries  = file_merged_new[file_merged_new["country-code"].isnull()]["name"].unique()

In [207]:
text = ""
for c in list_countries:
    if c not in country_to_delete:
        text += c +"||"
print(text)

In [ ]:
"United Korea (former)\n", => faire deux lignes North Korea, South Korea
"Korea, United" => faire deux lignes North Korea, South Korea
United Korea (former) => faire deux lignes
"Serbia and Montenegro" => faire deux lignes
"West Germany / East Germany => ???"
"North Yemen (former)", "South Yemen (former)" => voir ce que l'on fait
Yemen Arab Republic (Former)||Yemen Democratic (Former) => voir ce que l'on fait
"Eritrea and Ethiopia" => faire deux lignes,
"Serbia excluding Kosovo" => voir si y a Kosovo dans le fichier


In [194]:
replace_country = {
    "Saint-Pierre-et-Miquelon" : "Saint Pierre and Miquelon",
    "Cook Is" : "Cook Islands",
    "Central African Republic" : "Central African Rep.",
    'Congo, Rep.' : "Congo Rep.",
    "Yemen, Rep." : "Yemen Rep.",
    'Congo, Dem. Rep.' : "Congo Dem. Rep.",
    "USSR" : "Russia",
    "Kyrgyz" : "Kyrgyzstan",
    "Czechoslovakia" : "Czech Republic",
    "Czech Rep.": "Czech Republic",
    "Dominican Rep.": "Dominican Republic",
    "Korea, Dem. Rep." : "North Korea",
    "Korea, Rep." : "South Korea",
    "Falkland Islands (Malvinas)" : "Falkland Is (Malvinas)",
    "Lao" : "Laos",
    "Macao, China" : "Macao",
    "Macedonia, FYR" : "Macedonia",
    "Micronesia, Fed. Sts." : "Micronesia (Federated States of)",
    "St. Barthélemy": "Saint Barthélemy",
    "St. Lucia" : "Saint Lucia",
    "St. Martin" : "Saint Martin",
    "St. Martin (French part)" : "Saint Martin",
    "Hong Kong, China" : "Hong Kong",
    "Slovak Republic" : "Slovakia",
    "Somaliland" : "Somalia",
    "Virgin Islands, British" : "British Virgin Islands"
}

def get_right_name(country):
    if country in replace_country:
        return replace_country[country]
    else:
        return country

In [202]:
file_merged_rep_count = file_merged.copy()
file_merged_rep_count['name'] = file_merged_rep_count.apply(axis=1, func=lambda x: get_right_name(x['name']))

In [196]:
file_merged_rep_count.head()

,name,indicator,indicator2,year,value,new_name
0,Abkhazia,Aid received,"ODA received, total (constant 2010 US$)",1960,NaN,Abkhazia
1,Afghanistan,Aid received,"ODA received, total (constant 2010 US$)",1960,1.088000e+08,Afghanistan
2,Akrotiri and Dhekelia,Aid received,"ODA received, total (constant 2010 US$)",1960,NaN,Akrotiri and Dhekelia
3,Albania,Aid received,"ODA received, total (constant 2010 US$)",1960,NaN,Albania
4,Algeria,Aid received,"ODA received, total (constant 2010 US$)",1960,3.405040e+09,Algeria


In [203]:
file_merged_new_2 = pd.merge(file_merged_rep_count, df_world[['name', 'region', 'country-code']], on='name', how='inner')

In [208]:
len(file_merged_new_2)

239366

In [209]:
file_merged_new_2['name'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia',
       'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Rep.',
       'Chad', 'Chile', 'China', 'Christmas Island', 'Colombia', 'Comoros',
       'Congo Dem. Rep.', 'Congo Rep.', 'Cook Islands', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Ethiopia', 'Falkland Is (Malvinas)', 'Fiji', 'Finland'

In [213]:
file_merged_new_2.to_csv('ind_with_all_countries.csv', sep=";")

In [214]:
len(file_merged_new_2["indicator"].unique())

29

## Comparaison avec les pays du fichier GEO

In [215]:
import json

with open('./analyse_data/countries.geo.json') as data_file:    
    geo_country = json.load(data_file)



In [228]:
list_geo_country = []
for i in range(len(geo_country["features"])):
    list_geo_country.append(geo_country["features"][i]["properties"]["name"])

In [230]:
country_gini = ['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas, The', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Central African Rep.', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo Dem. Rep.', 'Congo Rep.', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominican Rep.', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guin', 'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'French Guiana', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Greenland', 'Guadeloupe', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Lithuania', 'Luxembourg', 'Macedonia', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Mauritania', 'Mauritius', 'Mexico', 'Moldova', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'North Korea', 'Norway', 'Oman', 'Pakistan', 'Palestine', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Reunion', 'Romania', 'Russia', 'Rwanda', 'Saint Lucia', 'SaoTome & Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Slovak Republic', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Korea', 'Spain', 'Sri Lanka', 'St. Kitts & Nevis', 'St. Lucia', 'Sudan', 'Suriname', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Rep', 'Taiwan, China', 'Tajikistan', 'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Uganda', 'Ukraine', 'United Arab Emir', 'United Kingdom', 'United States', 'Uruguay', 'Uzbekistan', 'Venezuela', 'Vietnam', 'Yemen Rep.', 'Yugoslavia', 'Zambia', 'Zimbabwe']
country_ko = []
for c in country_gini:
    if c not in list_geo_country:
        country_ko.append(c)